In [4]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tqdm import tqdm

file_dir = '../assets/combined_milk_final.csv'
time_dir = '../assets/time.csv'

### Functions

In [5]:
def price_discount(df, sku = None, year_col='Year'):
    out_df = df.copy()
    price_col = f'{sku}_Price' if sku is not None else 'Price'
    # Calculate the lower bound as 95% of the maximum price within each year
    lower_bound = out_df.groupby(year_col)[price_col].transform(lambda x: np.max(x) * 0.95)
    # Filter the DataFrame based on the condition
    filtered_df = out_df[out_df[price_col] >= lower_bound]
    # Calculate the median for each group of time_year in the filtered DataFrame
    median_by_time_year = filtered_df.groupby(year_col)[price_col].median()
    # Copy the median values back into the original DataFrame by year_col
    out_df.loc[:,'median_price'] = out_df[year_col].map(median_by_time_year)
    # Compute the relative discount
    pc_disc =  out_df['median_price'] / out_df[price_col]
    
    
    z_scores = (pc_disc - np.mean(pc_disc)) / np.std(pc_disc)
    # Identify indices where the absolute z-score is greater than or equal to 3
    outlier_indices = np.where(np.abs(z_scores) >= 3)[0]
    # Replace outliers with the maximum value from X_pc_d excluding those outliers
    if len(outlier_indices) > 0:
        pc_disc[outlier_indices] = np.max(pc_disc[~np.isin(np.arange(len(pc_disc)), outlier_indices)])


    # Update Price column
    out_df.loc[:, price_col] = pc_disc

    return out_df , np.mean(pc_disc), np.std(pc_disc)


def sales_lag(df, sku = None, neg = True):
    out_df = df.copy()
    sales_col = f'{sku}_Sales' if sku is not None else 'Sales'
    out_df[sales_col] = out_df[sales_col].shift(1) if neg is True else out_df[sales_col].shift(1)
    return out_df

def sum_columns(df, sku = None, promotype = 'Feature', neg = True):
    out_df = df.copy()
    columns_to_max = [col for col in out_df.columns if col.startswith(sku+'_'+promotype)] if sku is not None else [col for col in out_df.columns if col.startswith(promotype)]
    if not columns_to_max:
        return out_df
    # Calculate the maximum values using numpy
    sum_values = np.sum(out_df[columns_to_max].values, axis=1)
    # Create a new column with the maximum values
    max_column_name = f'{sku}_{promotype}' if neg is True else f'{promotype}'
    out_df[max_column_name] = sum_values if neg is True else sum_values
    # Drop the columns used in the max calculation
    out_df.drop(columns=columns_to_max, inplace=True)
    return out_df


def demand_coef(data, calendar, store_id_input, sku_id, window_size, train_year_from, train_year_to, test_year):

    store_df = data[data['Store_ID'] == store_id_input]
    store_sku_df = store_df[store_df['SKU'] == sku_id]


    # Get competitor df
    store_compet_sku_df = store_df[store_df['SKU'] != sku_id]

    # Competitor sku list
    compet_sku = store_df[store_df['SKU'] != sku_id]['SKU'].tolist()
    compet_sku = list(OrderedDict.fromkeys(compet_sku))

    store_compet_sku_df = store_compet_sku_df.pivot_table(index = ['Time_ID', 'Year', 'Store_ID'], columns = 'SKU', 
                                                        values = ['Price', 'Sales', 'Display1', 'Display2', 'Feature1', 'Feature2', 'Feature3', 'Feature4'])

    store_compet_sku_df.columns = ['_'.join([col[1], col[0]]) for col in store_compet_sku_df.columns]
    store_compet_sku_df.reset_index(inplace=True)


    store_compet_final = store_compet_sku_df.copy()
    for sku in compet_sku:
        store_compet_final, _, _ = price_discount(store_compet_final, sku)
        store_compet_final = sum_columns(store_compet_final, sku, 'Feature')    # Features
        store_compet_final = sum_columns(store_compet_final, sku, 'Display')    # Display
        store_compet_final = sales_lag(store_compet_final, sku)                 # Lag Sales 

    drop_cols = ['Store_ID', 'median_price']
    store_compet_final.drop(columns = drop_cols, inplace = True)

    store_sku_df.reset_index(drop = True, inplace= True)

    # SKU Features
    store_sku_df_part = store_sku_df.copy()
    store_sku_df_part, dis_mean, dis_std = price_discount(store_sku_df_part)                                                  # Price Column
    store_sku_df_part = sum_columns(store_sku_df_part, promotype = 'Feature', neg = False)                                    # Feature Column
    store_sku_df_part = sum_columns(store_sku_df_part, promotype = 'Display', neg = False)                                    # Display Column

    store_sku_df_part['Pricelag'] = store_sku_df_part['Price'].shift(1)                                                      # Price Lag Column
    store_sku_df_part['Featurelag'] = store_sku_df_part['Feature'].shift(1)                                                  #  Feature Lag Column
    store_sku_df_part['Displaylag'] = store_sku_df_part['Display'].shift(1)                                                  #  Display Lag Column

    store_sku_df_part['Saleslag'] = store_sku_df_part['Sales'].shift(1)                                                     # Log Sales Lag Column 
    store_sku_df_part['Sales_mov_avg'] = store_sku_df_part['Sales'].rolling(window = window_size).mean().shift(1)           # Log Sales Rolling Mean Lag Column
    store_sku_df_part['Sales'] = store_sku_df_part['Sales']                                                                 # Process Sales for Target Variable


    # Add Special Events
    event_col = ['Halloween', 'Thanksgiving', 'Christmas', 'NewYear', 'President', 'Easter', 'Memorial', '4thJuly', 'Labour']
    event_cols = [col if col == 'NewYear' else [col, f'{col}_1'] for col in event_col]
    event_cols = [item for sublist in event_cols for item in ([sublist] if isinstance(sublist, str) else sublist)]
    calendar_cols = calendar[['IRI Week']+ event_cols]
    calendar_cols = calendar_cols.fillna(0).astype(int)

    # Join Special Events to SKU Store Data
    store_sku_df_part = pd.merge(store_sku_df_part, calendar_cols, left_on='Time_ID', right_on='IRI Week', how='left')

    drop_cols = ['Discount','Store_ID', 'median_price', 'SKU', 'IRI Week']
    store_sku_df_part.drop(columns = drop_cols, inplace = True)

    # Train Models
    model_1 = Lasso(max_iter = 10000)
    model_2 = Lasso(max_iter = 10000)

    # Prepare training data for own + event features
    store_sku_part_trg = store_sku_df_part[(store_sku_df_part["Year"] >= train_year_from) & (store_sku_df_part["Year"] <= train_year_to)]
    store_sku_part_trg = store_sku_part_trg.iloc[window_size:]
    
    # Prepare training data for competitor sku features
    store_compet_trg = store_compet_final[(store_compet_final["Year"] >= train_year_from) & (store_sku_df_part["Year"] <= train_year_to)]
    store_compet_trg = store_compet_trg.iloc[window_size:]

    # Training Target variable
    sku_sales_train = store_sku_part_trg['Sales']

    # Feature Variables
    sku_train_drop = ['Time_ID', 'Year', 'Sales']
    compet_train_drop = ['Time_ID', 'Year']
    store_sku_part_trg = store_sku_part_trg.drop(columns = sku_train_drop)
    store_compet_trg = store_compet_trg.drop(columns = compet_train_drop)

    # Fit model 1
    model_1.fit(store_sku_part_trg, sku_sales_train)
    sku_sales_train_rsd = sku_sales_train - model_1.predict(store_sku_part_trg)
    # Fit model 2
    model_2.fit(store_compet_trg, sku_sales_train_rsd)

    # Save coefficients
    model_1_df = pd.DataFrame(model_1.coef_, index = store_sku_part_trg.columns, columns=[sku_id] )
    model_2_df = pd.DataFrame(model_2.coef_, index = store_compet_trg.columns, columns=[sku_id] )

    # Get bias term of both models
    bias_term = model_1.intercept_ + model_2.intercept_


    coef_df = pd.concat([model_1_df, model_2_df])

    # Prepare test data
    store_sku_part_test = store_sku_df_part[(store_sku_df_part["Year"] == test_year)]
    store_compet_test = store_compet_final[(store_compet_final["Year"] == test_year)]

    # Test Target variabe
    sku_sales_test = store_sku_part_test['Sales']
    store_sku_part_test = store_sku_part_test.drop(columns = sku_train_drop)
    store_compet_test = store_compet_test.drop(columns = compet_train_drop)

    full_predict = model_1.predict(store_sku_part_test) + model_2.predict(store_compet_test)
    full_predict = np.maximum(0, full_predict)

    mse =  mean_squared_error(sku_sales_test, full_predict)
    mape = mean_absolute_percentage_error(sku_sales_test, full_predict)
    mae = mean_absolute_error(sku_sales_test, full_predict)

    
    return coef_df, dis_mean, dis_std, bias_term, mse, mae, mape

In [6]:
data = pd.read_csv(file_dir)
calendar = pd.read_csv(time_dir)

store_id_input = 236117
window_size = 8
year_from = 2001
year_to = 2005
year_test = 2006
alphas = np.logspace(-4, 0, 100)

base_features = ['Price', 'Feature', 'Display', 'Pricelag', 'Featurelag', 'Displaylag', 
                 'Saleslag', 'Sales_mov_avg', 'Halloween', 'Halloween_1', 'Thanksgiving',
                 'Thanksgiving_1', 'Christmas', 'Christmas_1', 'NewYear', 'President',
                 'President_1', 'Easter', 'Easter_1', 'Memorial', 'Memorial_1', '4thJuly', '4thJuly_1', 'Labour', 'Labour_1']

unique_skus = list(data[data['Store_ID'] == store_id_input]['SKU'].unique())

new_index_parts = []
for code in unique_skus:
    new_index_parts.extend([
        f'{code}_Price',
        f'{code}_Display',
        f'{code}_Feature',
        f'{code}_Sales'
    ])

# Combine the base features with the new index parts
complete_index = base_features + new_index_parts

# Create an empty DataFrame with the specified list of row indexes
final_df = pd.DataFrame(index=complete_index)


df_z_score = pd.DataFrame(columns=["Mean", "Std_deviation", 'bias'])
df_metrics = pd.DataFrame(columns=['MSE', "MAE", "MAPE"])

for sku in tqdm(unique_skus):
    df_coef, mean, std, bias, mse, mae, mape = demand_coef(data, calendar, store_id_input, sku, window_size, year_from, year_to, year_test)
    final_df = final_df.join(df_coef, how = 'left').fillna(0)
    df_z_score.loc[sku] = [mean, std, bias]
    df_metrics.loc[sku] = [mse, mae, mape]


final_df.index = [idx.replace('Price', 'Discount') for idx in final_df.index]

100%|██████████| 36/36 [00:19<00:00,  1.81it/s]


In [7]:
# Export results
df_z_score_out = df_z_score.reset_index()
df_z_score_out = df_z_score_out.rename(columns={'index': 'SKU'})
df_z_score_out.to_csv('../assets/Z_scores.csv', index = False)
final_df.to_csv('../assets/Coefficients.csv')

In [8]:
df_metrics.to_csv('../assets/Demand_metrics.csv')

In [9]:
df_metrics['MAPE'].mean()

0.32467696837545384